In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [3]:
import visdom

vis = visdom.Visdom()
vis.close(env="main")


Setting up a new session...


''

In [16]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

In [5]:
device='cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device=='cuda':
  torch.cuda.manual_seed_all(777)

In [6]:
learning_rate=0.001
training_epochs=15
batch_size=32


In [7]:
mnist_train=dsets.MNIST(root='MNIST_data/',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)

mnist_test=dsets.MNIST(root='MNIST_data/',
                       train=False,
                       transform=transforms.ToTensor(),
                       download=True)

In [8]:
data_loader=torch.utils.data.DataLoader(dataset=mnist_train,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        drop_last=True)

In [9]:
class CNN(nn.Module):

  def __init__(self):
    super(CNN,self).__init__()
    self.layer1=nn.Sequential(
        nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.layer2=nn.Sequential(
        nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        nn.MaxPool2d(2)
    )

    self.layer3=nn.Sequential(
        nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.fc1=nn.Linear(3*3*128,625)
    self.relu=nn.ReLU()
    self.fc2=nn.Linear(625,10,bias=True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self,x):
    out=self.layer1(x)
    out=self.layer2(out)
    out=self.layer3(out)

    out=out.view(out.size(0),-1)
    out=self.fc1(out)
    out=self.relu(out)
    out=self.fc2(out)
    return out

In [10]:
model=CNN().to(device)

value=(torch.Tensor(1,1,28,28)).to(device)

print((model(value)).shape)

torch.Size([1, 10])


In [11]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [12]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

In [17]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished!')


[Epoch:1] cost = 0.031747061759233475
[Epoch:2] cost = 0.025501497089862823
[Epoch:3] cost = 0.0202800165861845
[Epoch:4] cost = 0.01711435243487358
[Epoch:5] cost = 0.014541278593242168
[Epoch:6] cost = 0.014471864327788353
[Epoch:7] cost = 0.01283595897257328
[Epoch:8] cost = 0.009982259944081306
[Epoch:9] cost = 0.011268292553722858
[Epoch:10] cost = 0.010874152183532715
[Epoch:11] cost = 0.008898891508579254
[Epoch:12] cost = 0.00923246517777443
[Epoch:13] cost = 0.007436058949679136
[Epoch:14] cost = 0.009123854339122772
[Epoch:15] cost = 0.009643478319048882
Learning Finished!


In [15]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean() 
    print('Accuracy:', accuracy.item())

C:\Users\user\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\user\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9715999960899353
